<a href="https://colab.research.google.com/github/Lucasvieirar/Aprendizado_de_m-quina_para_previs-o_de_tend-ncia_clim-tica-/blob/main/regressao_temperatura_bulbo_seco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd #carregamento da biblioteca
import plotly.express as px #carregamento da biblioteca - gráficos interativos
from sklearn.model_selection import train_test_split #carregamento da biblioteca - divisão de dados
from sklearn.neighbors import KNeighborsRegressor #carregamento da biblioteca - algoritmo de machine learning
from sklearn.tree import DecisionTreeRegressor #carregamento da biblioteca - algoritmo de machine learning
from sklearn.ensemble import RandomForestRegressor #carregamento da biblioteca - algoritmo de machine learning
from sklearn.metrics import r2_score #carregamento da biblioteca - métricas de desempenho
from sklearn.metrics import mean_absolute_error #carregamento da biblioteca  - métricas de desempenho
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_excel('/content/campo_grande_clima.xlsx')
df

,DATA,"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (KJ/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)"
0,2016-01-01,16.6,950.637500,950.941667,950.312500,420.416667,24.375000,21.495833,24.950000,23.804167,21.787500,21.112500,86.458333,80.791667,84.125000,78.125000,5.037500,2.166667
1,2016-01-02,10.0,951.245833,951.541667,950.908333,742.150000,24.745833,21.687500,25.454167,24.275000,22.112500,21.308333,86.208333,80.500000,83.708333,140.916667,5.495833,2.125000
2,2016-01-03,1.2,950.916667,951.233333,950.650000,637.095833,24.341667,21.600000,24.762500,23.975000,21.979167,21.225000,87.000000,82.791667,85.125000,160.833333,4.320833,1.925000
3,2016-01-04,19.0,951.000000,951.300000,950.670833,540.041667,24.908333,22.137500,25.550000,24.391667,22.533333,21.720833,87.541667,82.000000,85.125000,69.500000,3.812500,1.454167
4,2016-01-05,19.2,951.650000,951.975000,951.354167,391.762500,24.537500,21.929167,25.083333,24.029167,22.320833,21.495833,88.291667,82.625000,85.750000,116.541667,4.108333,1.904167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3283,2024-12-27,0.0,950.604167,950.804167,950.300000,1244.858333,24.958333,16.308333,25.466667,24.287500,16.904167,15.550000,63.125000,57.166667,60.083333,171.208333,5.554167,2.608333
3284,2024-12-28,0.0,952.183333,952.437500,951.937500,1275.904167,25.529167,15.854167,26.220833,24.758333,16.458333,15.029167,60.541667,53.125000,56.708333,158.500000,4.883333,2.562500
3285,2024-12-29,0.0,950.900000,951.187500,950.729167,1279.545833,25.545833,14.883333,26.366667,24.675000,15.591667,13.929167,57.875000,49.708333,53.958333,162.250000,4.541667,2.275000
3286,2024-12-30,0.0,949.095833,949.358333,948.845833,1190.966667,26.683333,14.950000,27.433333,25.845833,15.800000,14.125000,54.208333,47.041667,50.458333,185.541667,4.308333,2.245833


In [3]:
df = df.drop("DATA", axis=1)

ANALISE DE CORRELAÇÃO

In [4]:
corr_target = df.corr(numeric_only=True)['TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)']

print(corr_target.sort_values(ascending=False))

TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)             1.000000
TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)               0.998308
TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)               0.961841
TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)         0.405430
TEMPERATURA DO PONTO DE ORVALHO (°C)                     0.391928
TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)         0.377448
RADIACAO GLOBAL (KJ/m²)                                  0.292010
PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)          0.184763
PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)         0.184723
PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)    0.184023
VENTO, RAJADA MAXIMA (m/s)                               0.009055
VENTO, DIREÇÃO HORARIA (gr) (° (gr))                    -0.019870
VENTO, VELOCIDADE HORARIA (m/s)                         -0.036397
PRECIPITAÇÃO TOTAL, HORÁRIO (mm)                        -0.115557
UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)                -0.251266
UMIDADE RE

KNN

In [5]:
columns_to_drop = [
    'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)',
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)",
    'VENTO, DIREÇÃO HORARIA (gr) (° (gr))',
    'PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)',
    'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)',
    'PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)',
    'RADIACAO GLOBAL (KJ/m²)',
    'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)',
    'PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)',
    'VENTO, RAJADA MAXIMA (m/s)',
    'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)',
    'VENTO, VELOCIDADE HORARIA (m/s)',
    "TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)",
    "UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)",
    "TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)",
    "UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)",
    "TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)",
    "UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)",
    "TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)",
    "UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)",

    ]

x = df.drop(columns=columns_to_drop, axis=1)
y = df['TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)']



In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(x)

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [12]:
y_train.info()

<class 'pandas.core.series.Series'>
Index: 2301 entries, 2586 to 3174
Series name: TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)
Non-Null Count  Dtype  
--------------  -----  
2301 non-null   float64
dtypes: float64(1)
memory usage: 36.0 KB


In [13]:
knn = KNeighborsRegressor()
knn.fit(x_train, y_train)

KNeighborsRegressor()

In [14]:
y_pred = knn.predict(x_test)
y_pred

array([26.82      , 24.07416667, 13.1685    , 23.        , 19.96      ,
       27.71      , 21.52916667, 27.8       , 14.97316667, 26.5525    ,
       24.25416667, 22.02083333, 28.16166667, 26.35      , 20.94083333,
       24.06166667, 25.86083333, 24.57333333, 29.27833333, 26.43333333,
       27.27083333, 21.32083333, 21.42166667, 23.        , 26.24      ,
       23.6375    , 24.35666667, 23.98916667, 14.29666667, 23.49333333,
       24.315     , 25.11166667, 25.91333333, 27.44166667, 22.6525    ,
       22.99      , 26.14416667, 24.70916667, 25.4325    , 26.02416667,
       25.86416667, 21.62833333, 19.42833333, 21.38666667, 24.00416667,
       15.26833333, 23.63416667, 22.30416667, 26.64333333, 19.19166667,
       24.27166667, 23.50416667, 22.76083333, 26.98833333, 25.3025    ,
       17.87416667, 24.5075    , 23.60666667, 25.06416667, 27.31416667,
       20.0575    , 23.40166667, 23.75416667, 24.235     , 24.56      ,
       21.905     , 18.39916667, 28.92916667, 22.0475    , 22.43

In [15]:
# Métricas de desempenho R^2 (0 - 1 - quanto mais próximo de um melhor) | Métrica do erro absoluto (0 - infinito - quanto menor melhor)
def evalute_regression_model(model, x_test, y_test):
  pred = model.predict(x_test)
  print("R Square", r2_score(y_test, pred))
  print("MAE", mean_absolute_error(y_test, pred))

In [16]:
evalute_regression_model(knn, x_test, y_test)

R Square 0.9687129689176954
MAE 0.2595650101280586


In [17]:
fig = px.scatter(x=y_test, y=y_pred, title="Real x predito", trendline="ols",  labels={"x":"Real", "y":"Previsto"})
fig.show()

árvore de decisão

In [7]:
dt = DecisionTreeRegressor()
dt.fit(x_train, y_train)

DecisionTreeRegressor()

In [8]:
pred = dt.predict(x_test)
pred

array([26.62916667, 23.99583333,  9.50416667, 23.        , 20.5125    ,
       27.80833333, 21.23333333, 27.3625    , 14.51666667, 26.175     ,
       24.44166667, 21.6625    , 28.48333333, 26.125     , 20.99166667,
       23.95416667, 25.9375    , 24.64583333, 29.25416667, 26.33333333,
       27.29166667, 20.97083333, 21.75416667, 23.        , 26.975     ,
       23.65416667, 24.56666667, 23.8       , 15.03333333, 23.74166667,
       24.25      , 25.66666667, 26.34166667, 27.525     , 22.5875    ,
       22.87083333, 26.2625    , 24.92916667, 25.45      , 24.8875    ,
       25.82083333, 20.6625    , 19.36666667, 21.26666667, 24.52083333,
       15.19166667, 23.65      , 22.625     , 26.74166667, 18.5125    ,
       24.        , 22.9125    , 22.7       , 27.22916667, 25.43333333,
       17.34583333, 24.475     , 23.4875    , 25.22083333, 27.45416667,
       20.30416667, 23.50416667, 24.35416667, 24.35416667, 24.5375    ,
       21.94583333, 18.125     , 29.4375    , 21.96666667, 22.12

In [9]:
print("R square score", r2_score(y_test, pred))
print("MAE", mean_absolute_error(y_test, pred))

R square score 0.9758031229817695
MAE 0.21113086964228936


In [10]:
fig = px.scatter(x=y_test, y=pred, title = 'Real x Previsto', trendline='ols', labels={"x":"Real", "y":"previsto"})
fig.show() #outilnes valores discrepantes

In [7]:
rf = RandomForestRegressor()
rf.fit(x_train, y_train)

RandomForestRegressor()

In [8]:
pred = rf.predict(x_test)
pred

array([26.72833333, 23.93070833, 10.65884167, 23.        , 20.54141667,
       27.827875  , 21.40029167, 27.12775   , 14.05925   , 26.25533333,
       24.34625   , 21.66041667, 28.37575   , 26.25866667, 20.79720833,
       23.94975   , 25.87075   , 24.584875  , 29.30608333, 26.548375  ,
       27.20466667, 21.022125  , 21.66270833, 23.        , 27.00879167,
       23.58416667, 24.52845833, 23.83820833, 14.60725   , 23.75079167,
       24.392625  , 25.56645833, 25.90075   , 27.506125  , 22.65783333,
       23.06004167, 26.18179167, 24.84045833, 25.44483333, 24.48729167,
       25.81495833, 20.676     , 19.4375    , 21.43575   , 24.32945833,
       15.21288333, 23.71641667, 22.446     , 26.69416667, 18.70454167,
       24.00854167, 22.87829167, 22.69454167, 27.21891667, 25.41491667,
       17.42695   , 24.45370833, 23.56766667, 25.175     , 27.46341667,
       20.36270833, 23.41608333, 24.09229167, 24.292375  , 24.56179167,
       21.90375   , 18.20191667, 29.527     , 21.85941667, 22.29

In [9]:
print("R square score", r2_score(y_test, pred))
print("MAE", mean_absolute_error(y_test, pred))

R square score 0.9799595747069818
MAE 0.16285310030760528


In [10]:
!pip install -U plotly>=6.1.1

In [10]:
fig = px.scatter(x=y_test, y=pred, title = 'Real x Previsto', trendline='ols', labels={"x":"Real", "y":"previsto"})
fig.show()